In [7]:
from scipy.io import loadmat
import numpy as np

#SETUP
X = loadmat("RawData.mat")['X']
y = loadmat("RawData.mat")['y']
Xones = np.ones((len(X),1))

#Single threshold for classification
thresh = 1500
for i in range(len(X)):
    if(y[i] <= thresh):
        y[i] = -1
    if(y[i] > thresh):
        y[i] = 1

print(np.mean(y))

#Optional: Eliminate high values
#for i in range(110):
#    X = np.delete(X, np.argmax(y), 0)
#    y = np.delete(y, np.argmax(y), 0)
#    X = np.delete(X, np.argmin(y), 0)
#    y = np.delete(y, np.argmin(y), 0)

TwoNormCol = np.zeros((len(X.T),1))

#Remove keyword columns (poorly treated data)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)
X = np.delete(X, 17, 1)

#Normalize columns to 2-norm
for i in range(len(X.T)):
    TwoNormCol[i] = np.sqrt(X[i,:]@X[i,:])
    X[i,:] = X[i,:]/TwoNormCol[i]
#print(TwoNormCol)
#print(X[0,:])
#print(y)
    
#Form subsets (indices, first group is full X)
Xsubs = np.array([[0,49],[0,5],[5,7],[7,9],[9,11],[11,17],[17,20],[20,26],[28,33],[33,37],[37,45],[45,49]])

#Create and run over 11 sets of 3604 entries from X and y,
#using 1 as a primary test set (when needed for lambda)
setArr = np.array([[0,int(len(X)/11)],[int(len(X)/11),int(2*len(X)/11)],[int(2*len(X)/11),int(3*len(X)/11)],\
                   [int(3*len(X)/11),int(4*len(X)/11)],[int(4*len(X)/11),int(5*len(X)/11)],\
                   [int(5*len(X)/11),int(6*len(X)/11)],[int(6*len(X)/11),int(7*len(X)/11)],\
                   [int(7*len(X)/11),int(8*len(X)/11)],[int(8*len(X)/11),int(9*len(X)/11)],\
                   [int(9*len(X)/11),int(10*len(X)/11)],[int(10*len(X)/11),int(len(X))]])
#print(setArr)

#Set up network layer count
networkCount = 5
networkTest = np.linspace(2, 6, networkCount)
#print(networkCount)

#Error tally storage
errorOrig = np.zeros((110,1))


################################3

#Only looping over full X matrix
Xset = 0
   
for i in range(11):
    for j in range(11):
         if i != j:
            print("X subset = ",Xset,", i = ",i,", j = ",j)
            testTally = 0
            #Set up Training and testing sets
            for k in range(11):
                if i == k:
                    XTest1 = X[setArr[i,0]:setArr[i,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                    #print(XTest1)
                    yTest1 = y[setArr[i,0]:setArr[i,1]]
                    #print(yTest1)
                if j == k:
                    XTest2 = X[setArr[j,0]:setArr[j,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                    #print(XTest2)
                    yTest2 = y[setArr[j,0]:setArr[j,1]]
                    #print(yTest2)
                if k != j and k != i:
                    if testTally == 0:
                        XTrain = X[setArr[k,0]:setArr[k,1],Xsubs[Xset,0]:Xsubs[Xset,1]]
                        yTrain = y[setArr[k,0]:setArr[k,1]]
                        testTally = 1
                    else:
                        XTrain = np.concatenate((XTrain, X[setArr[k,0]:setArr[k,1],Xsubs[Xset,0]:Xsubs[Xset,1]]),\
                                                axis=0)
                        yTrain = np.concatenate((yTrain, y[setArr[k,0]:setArr[k,1]]), axis=0)

    
                # Storage for current iteration
                WNET = np.zeros((len(X.T),networkCount))
                
                # Loop through different counts
                for index in range(networkCount):
                    M = networkTest[index]
                    WNET[:,index] = 1
                    
                    
                    
                    
                    
                    
    
                #check with Test Set 1
                y_net1 = XTest1@WNET
                errorTallyLasso = np.zeros((lambdaCount,1))
                #errorTallyRid = np.zeros((lambdaCount,1))
                for lam in range(lambdaCount):
                    for index in range(len(XTest1)):
                        if (yTest1[index] > 0 and y_lasso1[index,lam] < 0) or (yTest1[index] < 0 and y_lasso1[index,lam] > 0 or y_lasso1[index,lam] == 0):
                            errorTallyLasso[lam] = errorTallyLasso[lam] + 1
                    #if (max(y_lasso1[:,lam]) < 1e-6):
                    #    print(max(y_lasso1[:,lam]))
                    #print(errorTallyLasso[lam])

                #print(errorTallyLasso)
                #print(errorTallyRid)
                OptimalLambdaLasso = np.argmin(errorTallyLasso)
                #print("Optimal lambda: ",lambdaTest[OptimalLambdaLasso]," with min ",min(errorTallyLasso)/(len(X)/11))
                #print(np.argmin(errorTallyLasso))
                #OptimalLambdaRid = np.argmin(errorTallyRid)
                #print(np.argmin(errorTallyRid))
                
                #Running on Testing 2
                wOrig = np.linalg.inv(XTrain.T@XTrain)@XTrain.T@yTrain
                yTestOrig = XTest2@wOrig
                #print(yTestOrig)
                yTestLASSO = XTest2@WLASSO[:,OptimalLambdaLasso]

                errorOrigTemp = 0
                errorLASSOTemp = 0
                for index in range(len(XTest2)):
                    if (yTest1[index] > 0 and yTestOrig[index] < 0) or (yTest1[index] < 0 and yTestOrig[index] > 0):
                        errorOrigTemp = errorOrigTemp + 1
                    if (yTest1[index] > 0 and yTestLASSO[index] < 0) or (yTest1[index] < 0 and yTestLASSO[index] > 0):
                        errorLASSOTemp = errorLASSOTemp + 1
                
                errorOrigTemp = errorOrigTemp/len(XTest2)
                errorLASSOTemp = errorLASSOTemp/len(XTest2)
                print("Mean error from standard least squares: ",errorOrigTemp)
                print("Mean error from LASSO: ",errorLASSOTemp)
                
                errorOrig[Xset] = errorOrig[Xset] + errorOrigTemp
                errorLASSO[Xset] = errorLASSO[Xset] + errorLASSOTemp

GlobalMeanOrigError = np.zeros((11,1))
GlobalMeanLASSOError = np.zeros((11,1))
for index in range(11):
    GlobalMeanOrigError[index] = errorOrig[index]/110
    GlobalMeanLASSOError[index] = errorLASSO[index]/110
    print("Mean Error for pure least squares, X subset ",(index+1)," = ",GlobalMeanOrigError[index])
    print("Mean Error for LASSO, X subset ",(index+1)," = ",GlobalMeanLASSOError[index])





-0.07986076077086066
X subset =  0 , i =  0 , j =  1


NameError: name 'lambdaCount' is not defined